<a href="https://colab.research.google.com/github/itihub/jupyter_learn/blob/main/supply_chain_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 供应链网络优化模型

本 Notebook 旨在构建一个快消品供应链网络优化模型，目标是最小化总成本（生产、运输、库存和惩罚成本）。模型考虑了工厂、分仓、客户之间的多层级关系，以及多周期生产和带有最小/最大产值约束的生产线策略。

我们将使用 `pulp` 库进行线性规划，并使用 `pandas`, `plotly` 和 `folium` 进行数据分析和可视化。


强大的决策支持工具，可以从中挖掘出远超“最优解”本身的商业价值数据和风控数据。以下是你可以从模型结果中提取和分析的一些关键洞察，这些都对商业决策和风险管理至关重要：
1. 商业价值数据
这些数据可以帮助你优化运营策略，提高利润率。

+ 资源利用率：
    + 生产线利用率：通过计算 value(produce) 除以生产线的最大产能，你可以知道每条生产线在每个周期的繁忙程度。这有助于你决定是否需要升级、停用或投资新生产线。
    + 仓储空间利用率：通过分析 value(factory_inventory) 和 value(warehouse_inventory)，你可以计算出库存的平均占用率。这有助于评估现有仓储能力是否足够，或者是否需要调整租金、寻找新仓库。

+ 成本效率分析：
    + 单位成本：模型可以计算出每件产品从生产到最终交付给客户的总单位成本。这可以作为定价策略的依据，并帮助你识别成本最高的配送路径或生产线。
    + 成本分解细化：你已经实现了总成本的分解。但你可以更进一步，按产品、按客户或按周期来分解成本，找出哪些产品或客户是利润的主要来源，或者哪些时期成本最高。

+ 库存策略优化：
    + 安全库存量：通过分析模型中的库存水平，你可以了解在满足所有需求的前提下，模型如何管理库存。这可以帮助你设定合理的安全库存量，以平衡库存成本和缺货风险。
    + 提前生产策略：如果模型在需求较低的周期安排了大量生产，并将产品存入仓库，这说明模型认为提前生产比在需求高峰期紧急生产更经济。你可以将此作为季节性或周期性生产的指导方针。

2. 风控数据
这些数据可以帮助你评估和管理供应链中的潜在风险，为制定应急预案提供依据。

+ 单点依赖风险：
    + 工厂生产集中度：分析 produce 变量，如果某个工厂承担了绝大多数产品的生产，那么该工厂一旦停工，将对整个供应链造成巨大冲击。模型可以量化这种集中度。
    + 配送路径集中度：分析 flow 变量，如果某个分仓或运输路线承担了绝大部分的配送任务，那么这条路径的任何中断（如交通管制、自然灾害）都可能导致大规模延误。

+ 需求满足风险：
    + 缺货惩罚成本：你的模型包含了 penalty_cost_val。这个值如果非零，就意味着模型在最优解中允许了部分缺货，这本身就是一种风险信号。你可以通过分析是哪个客户、哪个周期、哪种产品出现了缺货，来识别最脆弱的环节。

+ 产能瓶颈识别：
    + 最大生产量约束：如果某个工厂的生产量 value(produce) 总是接近其最大产能约束，这表明该工厂是整个供应链的瓶颈。你可以将此作为投资扩建或寻找备选供应商的决策依据。

In [ ]:
!pip install pulp
!pip install pandas
!pip install plotly
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 34.1 MB/s eta 0:00:00


In [ ]:
# 导入所需库
from pulp import *
import pandas as pd
from collections import defaultdict
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, Fullscreen, PolyLineTextPath
from IPython.display import display


## 数据准备

In [12]:
# --- 1. 数据准备 ---

# 定义所有站点、产品和周期
factories = ['北京工厂', '广州工厂', '上海工厂']
warehouses = ['上海分仓', '武汉分仓', '成都分仓']
customers = ['沈阳客户', '深圳客户', '成都客户', '西安客户', '杭州客户']
periods = [1, 2, 3]  # 三旬
products = ['橙汁', '苹果汁', '葡萄汁', '芒果汁']
# 产品数据（原料成本、售价、重量、体积）
product_data = {
    '橙汁': {'raw_material_cost': 1.5, 'sales_price': 15.0, 'weight': 1.2, 'volume': 0.001},
    '苹果汁': {'raw_material_cost': 1.2, 'sales_price': 13.0, 'weight': 1.1, 'volume': 0.0009},
    '葡萄汁': {'raw_material_cost': 1.8, 'sales_price': 18.0, 'weight': 1.3, 'volume': 0.0011},
    '芒果汁': {'raw_material_cost': 2.0, 'sales_price': 20.0, 'weight': 1.4, 'volume': 0.0012}
}

# 生产线信息
production_lines_info = {
    '北京工厂': {
        'Line_1': {
            'products': ['橙汁', '葡萄汁'],
            'min_daily_output': {'橙汁': 2000, '葡萄汁': 1800},
            'max_daily_output': {'橙汁': 5000, '葡萄汁': 4500}
        },
        'Line_2': {
            'products': ['苹果汁', '芒果汁'],
            'min_daily_output': {'苹果汁': 1500, '芒果汁': 1200},
            'max_daily_output': {'苹果汁': 4000, '芒果汁': 3000}
        }
    },
    '广州工厂': {
        'Line_A': {
            'products': ['橙汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1500, '芒果汁': 1000},
            'max_daily_output': {'橙汁': 6000, '芒果汁': 5000}
        },
        'Line_B': {
            'products': ['苹果汁', '葡萄汁'],
            'min_daily_output': {'苹果汁': 1800, '葡萄汁': 1600},
            'max_daily_output': {'苹果汁': 6500, '葡萄汁': 5500}
        }
    },
    '上海工厂': {
        'Line_X': {
            'products': ['橙汁', '苹果汁', '葡萄汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1000, '苹果汁': 800, '葡萄汁': 900, '芒果汁': 700},
            'max_daily_output': {'橙汁': 4000, '苹果汁': 3500, '葡萄汁': 3800, '芒果汁': 3000}
        }
    }
}

all_lines = list(set([line for f in factories for line in production_lines_info[f]]))


# 成本数据 (单位: 元/件)
production_cost = {
    ('北京工厂', 'Line_1', '橙汁'): 8, ('北京工厂', 'Line_1', '葡萄汁'): 9,
    ('北京工厂', 'Line_2', '苹果汁'): 7.8, ('北京工厂', 'Line_2', '芒果汁'): 9.5,
    ('广州工厂', 'Line_A', '橙汁'): 7.5, ('广州工厂', 'Line_A', '芒果汁'): 9.2,
    ('广州工厂', 'Line_B', '苹果汁'): 7, ('广州工厂', 'Line_B', '葡萄汁'): 8.5,
    ('上海工厂', 'Line_X', '橙汁'): 8.2, ('上海工厂', 'Line_X', '苹果汁'): 7.5,
    ('上海工厂', 'Line_X', '葡萄汁'): 8.8, ('上海工厂', 'Line_X', '芒果汁'): 9.8
}

# 生产线激活成本 (单位: 元/旬)
line_activation_cost = {
    ('北京工厂', 'Line_1'): 5000, ('北京工厂', 'Line_2'): 4800,
    ('广州工厂', 'Line_A'): 4500, ('广州工厂', 'Line_B'): 4000,
    ('上海工厂', 'Line_X'): 6000
}


# 仓储成本
factory_inventory_cost = {
    '北京工厂': {'橙汁': 0.08, '苹果汁': 0.10, '葡萄汁': 0.09, '芒果汁': 0.11},
    '广州工厂': {'橙汁': 0.07, '苹果汁': 0.09, '葡萄汁': 0.08, '芒果汁': 0.10},
    '上海工厂': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12}
}

warehouse_inventory_cost = {
    '上海分仓': {'橙汁': 0.1, '苹果汁': 0.12, '葡萄汁': 0.11, '芒果汁': 0.13},
    '武汉分仓': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12},
    '成都分仓': {'橙汁': 0.11, '苹果汁': 0.13, '葡萄汁': 0.12, '芒果汁': 0.14}
}

# 运输成本 (部分示例，实际应用中需完善所有路径)
transport_cost_fw = {
    ('北京工厂', '上海分仓'): 0.5, ('北京工厂', '武汉分仓'): 0.6, ('北京工厂', '成都分仓'): 0.9,
    ('广州工厂', '上海分仓'): 0.4, ('广州工厂', '武汉分仓'): 0.3, ('广州工厂', '成都分仓'): 0.7,
    ('上海工厂', '上海分仓'): 0.1, ('上海工厂', '武汉分仓'): 0.5, ('上海工厂', '成都分仓'): 0.8
}
transport_cost_fc = {
    ('北京工厂', '沈阳客户'): 0.2, ('广州工厂', '沈阳客户'): 1.5, ('上海工厂', '沈阳客户'): 0.8,
    ('北京工厂', '深圳客户'): 1.8, ('广州工厂', '深圳客户'): 0.2, ('上海工厂', '深圳客户'): 0.7,
    ('北京工厂', '成都客户'): 1.2, ('广州工厂', '成都客户'): 0.9, ('上海工厂', '成都客户'): 0.4,
    ('北京工厂', '西安客户'): 0.7, ('广州工厂', '西安客户'): 1.1, ('上海工厂', '西安客户'): 0.6,
    ('北京工厂', '杭州客户'): 0.9, ('广州工厂', '杭州客户'): 0.8, ('上海工厂', '杭州客户'): 0.3
}
transport_cost_wc = {
    ('上海分仓', '沈阳客户'): 0.8, ('武汉分仓', '沈阳客户'): 1.3, ('成都分仓', '沈阳客户'): 1.5,
    ('上海分仓', '深圳客户'): 0.6, ('武汉分仓', '深圳客户'): 0.7, ('成都分仓', '深圳客户'): 1.0,
    ('上海分仓', '成都客户'): 0.5, ('武汉分仓', '成都客户'): 0.4, ('成都分仓', '成都客户'): 0.2,
    ('上海分仓', '西安客户'): 0.3, ('武汉分仓', '西安客户'): 0.5, ('成都分仓', '西安客户'): 0.3,
    ('上海分仓', '杭州客户'): 0.2, ('武汉分仓', '杭州客户'): 0.6, ('成都分仓', '杭州客户'): 0.9
}


# 客户需求数据 (示例，需根据实际情况补充所有客户和周期)
demand = {
    '沈阳客户': {'橙汁': {1: 10000, 2: 12000, 3: 8000, 4: 9000}, '苹果汁': {1: 5000, 2: 6000, 3: 7000, 4: 6500},
               '葡萄汁': {1: 3000, 2: 4000, 3: 3500, 4: 3800}, '芒果汁': {1: 2000, 2: 2500, 3: 2200, 4: 2400}},
    '深圳客户': {'橙汁': {1: 15000, 2: 18000, 3: 16000, 4: 17000}, '苹果汁': {1: 8000, 2: 9000, 3: 10000, 4: 9500},
               '葡萄汁': {1: 6000, 2: 7000, 3: 6500, 4: 6800}, '芒果汁': {1: 5000, 2: 5500, 3: 5200, 4: 5800}},
    '成都客户': {'橙汁': {1: 12000, 2: 10000, 3: 14000, 4: 13000}, '苹果汁': {1: 7000, 2: 6000, 3: 8000, 4: 7500},
               '葡萄汁': {1: 5000, 2: 4500, 3: 5500, 4: 5200}, '芒果汁': {1: 4000, 2: 3500, 3: 4200, 4: 4100}},
    '西安客户': {'橙汁': {1: 8000, 2: 9000, 3: 7000, 4: 8500}, '苹果汁': {1: 4000, 2: 4500, 3: 3800, 4: 4200},
               '葡萄汁': {1: 3000, 2: 3200, 3: 2800, 4: 3100}, '芒果汁': {1: 2500, 2: 2800, 3: 2400, 4: 2600}},
    '杭州客户': {'橙汁': {1: 9000, 2: 8500, 3: 9500, 4: 9200}, '苹果汁': {1: 5000, 2: 4800, 3: 5200, 4: 5100},
               '葡萄汁': {1: 4000, 2: 3800, 3: 4200, 4: 4100}, '芒果汁': {1: 3000, 2: 2800, 3: 3200, 4: 3100}}
}


# 惩罚成本
penalty_cost_per_unit = 100

# 地理位置数据 (用于地图可视化)
locations = {
    '北京工厂': (39.9042, 116.4074), '广州工厂': (23.1291, 113.2644), '上海工厂': (31.2304, 121.4737),
    '上海分仓': (31.2304, 121.4737), '武汉分仓': (30.5928, 114.3055), '成都分仓': (30.5728, 104.0668),
    '沈阳客户': (41.7963, 123.4315), '深圳客户': (22.5431, 114.0579), '成都客户': (30.5728, 104.0668),
    '西安客户': (34.2632, 108.9525), '杭州客户': (30.2500, 120.1600)
}

## 数据建模

In [13]:
# --- 2. 模型构建与求解 ---

"""构建并求解供应链优化模型"""
model = LpProblem("Supply_Chain_Optimization", LpMinimize)
days_per_period = 10

# 决策变量
line_on = LpVariable.dicts("Line_On", (factories, all_lines, products, periods), 0, 1, LpBinary)
produce = LpVariable.dicts("Produce", (factories, all_lines, products, periods), 0, None, LpContinuous)
flow_fw = LpVariable.dicts("Flow_FW", (factories, warehouses, products, periods), 0, None, LpContinuous)
flow_fc = LpVariable.dicts("Flow_FC", (factories, customers, products, periods), 0, None, LpContinuous)
flow_wc = LpVariable.dicts("Flow_WC", (warehouses, customers, products, periods), 0, None, LpContinuous)
factory_inventory = LpVariable.dicts("Factory_Inventory", (factories, products, periods), 0, None, LpContinuous)
warehouse_inventory = LpVariable.dicts("Warehouse_Inventory", (warehouses, products, periods), 0, None, LpContinuous)

# 目标函数：最小化总成本
total_cost = []
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    total_cost.append(production_cost[(f, line, p)] * produce[f][line][p][t])
                    total_cost.append(line_activation_cost[(f, line)] * line_on[f][line][p][t])
for f in factories:
    for p in products:
        for t in periods:
            total_cost.append(factory_inventory_cost[f][p] * factory_inventory[f][p][t])
for w in warehouses:
    for p in products:
        for t in periods:
            total_cost.append(warehouse_inventory_cost[w][p] * warehouse_inventory[w][p][t])
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fw[(f, w)] * flow_fw[f][w][p][t])
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fc.get((f, c), 0) * flow_fc[f][c][p][t])
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_wc.get((w, c), 0) * flow_wc[w][c][p][t])
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = LpVariable(f"Unmet_Demand_{c}_{p}_{t}", 0, None)
            received_flow = lpSum([flow_fc[f][c][p][t] for f in factories]) + lpSum([flow_wc[w][c][p][t] for w in warehouses])
            model += unmet_demand_var >= demand[c][p][t] - received_flow
            total_cost.append(penalty_cost_per_unit * unmet_demand_var)

model += lpSum(total_cost)

# 约束条件
for c in customers:
    for p in products:
        for t in periods:
            model += lpSum([flow_fc[f][c][p][t] for f in factories] + [flow_wc[w][c][p][t] for w in warehouses]) >= demand[c][p][t]

for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    min_val = production_lines_info[f][line]['min_daily_output'][p] * days_per_period
                    max_val = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                    model += produce[f][line][p][t] >= min_val * line_on[f][line][p][t]
                    model += produce[f][line][p][t] <= max_val * line_on[f][line][p][t]
                else:
                    model += produce[f][line][p][t] == 0
                    model += line_on[f][line][p][t] == 0

for f in factories:
    for line in production_lines_info[f]:
        for t in periods:
            model += lpSum([line_on[f][line][p][t] for p in products]) <= 1

for f in factories:
    for p in products:
        for t in periods:
            total_produced = lpSum([produce[f][line][p][t] for line in production_lines_info[f]])
            total_shipped = lpSum([flow_fw[f][w][p][t] for w in warehouses] + [flow_fc[f][c][p][t] for c in customers])
            if t == periods[0]:
                model += factory_inventory[f][p][t] == total_produced - total_shipped
            else:
                model += factory_inventory[f][p][t] == factory_inventory[f][p][t-1] + total_produced - total_shipped

for w in warehouses:
    for p in products:
        for t in periods:
            inflow = lpSum([flow_fw[f][w][p][t] for f in factories])
            outflow = lpSum([flow_wc[w][c][p][t] for c in customers])
            if t == periods[0]:
                model += warehouse_inventory[w][p][t] == inflow - outflow
            else:
                model += warehouse_inventory[w][p][t] == warehouse_inventory[w][p][t-1] + inflow - outflow

# 求解模型
model.solve()
if LpStatus[model.status] != 'Optimal':
    print("模型求解失败，未找到最优解。")

## 成本可视化

In [14]:
# --- 3. 结果分析与可视化 ---

"""分析模型结果并进行可视化"""
print(f"总成本：{value(model.objective):,.2f} 元")

# 提取成本数据
# 生产成本
prod_cost_val = 0
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if value(produce[f][line][p][t]) > 0:
                    prod_cost_val += value(produce[f][line][p][t]) * production_cost[(f, line, p)]

# 生产线激活成本
act_cost_val = 0
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if value(line_on[f][line][p][t]) > 0:
                    act_cost_val += value(line_on[f][line][p][t]) * line_activation_cost[(f, line)]

# 运输成本
transport_cost_val = 0
# 工厂到分仓
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                if value(flow_fw[f][w][p][t]) > 0:
                    transport_cost_val += value(flow_fw[f][w][p][t]) * transport_cost_fw.get((f, w), 0)
# 工厂到客户
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_fc[f][c][p][t]) > 0:
                    transport_cost_val += value(flow_fc[f][c][p][t]) * transport_cost_fc.get((f, c), 0)
# 分仓到客户
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_wc[w][c][p][t]) > 0:
                    transport_cost_val += value(flow_wc[w][c][p][t]) * transport_cost_wc.get((w, c), 0)

# 工厂库存成本
factory_inv_cost_val = 0
for f in factories:
    for p in products:
        for t in periods:
            if value(factory_inventory[f][p][t]) > 0:
                factory_inv_cost_val += value(factory_inventory[f][p][t]) * factory_inventory_cost[f][p]

# 分仓库存成本
warehouse_inv_cost_val = 0
for w in warehouses:
    for p in products:
        for t in periods:
            if value(warehouse_inventory[w][p][t]) > 0:
                warehouse_inv_cost_val += value(warehouse_inventory[w][p][t]) * warehouse_inventory_cost[w][p]

# 惩罚成本
penalty_cost_val = 0
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = model.variablesDict()[f"Unmet_Demand_{c}_{p}_{t}"]
            if value(unmet_demand_var) > 0:
                penalty_cost_val += value(unmet_demand_var) * penalty_cost_per_unit
    cost_breakdown = {
        '生产成本': prod_cost_val,
        '运输成本': transport_cost_val,
        '库存成本': factory_inv_cost_val + warehouse_inv_cost_val,
        '生产线激活成本': act_cost_val,
        '惩罚成本': penalty_cost_val
    }

# 可视化1: 成本分解饼图
cost_df = pd.DataFrame(list(cost_breakdown.items()), columns=['Cost Type', 'Amount'])
fig1 = px.pie(cost_df, values='Amount', names='Cost Type', title='总成本分解')
fig1.show()


总成本：3,225,176.00 元


## 供应链的生产-需求平衡

In [15]:
# 1. 提取工厂每旬每种产品的产量
production_data_fine = []
for f in factories:
    for p in products:
        for t in periods:
            total_produced = sum(
                value(produce[f][line][p][t])
                for line in production_lines_info[f]
            )
            if total_produced > 0:
                production_data_fine.append({
                    '地点': f,
                    '周期（旬）': t,
                    '产品': p,
                    '数量': total_produced,
                    '类型': '产量'
                })

# 2. 提取客户每旬每种产品的需求量
demand_data_fine = []
for c in customers:
    for p in products:
        for t in periods:
            total_demanded = demand[c][p][t]
            if total_demanded > 0:
                demand_data_fine.append({
                    '地点': c,
                    '周期（旬）': t,
                    '产品': p,
                    '数量': total_demanded,
                    '类型': '需求量'
                })

# 合并生产和需求细化数据
production_demand_fine_df = pd.DataFrame(production_data_fine + demand_data_fine)

print("\n--- 工厂/客户产量与需求对照（细化到产品） ---")
if not production_demand_fine_df.empty:
    display(production_demand_fine_df)

    # --- 可视化：细化到产品的产量与需求对照 ---
    # 使用 facet_col 来按地点切分，使用 color 来区分产品
    fig = px.bar(
        production_demand_fine_df,
        x='周期（旬）',
        y='数量',
        color='产品',
        facet_col='地点',
        barmode='group',
        title='细化到产品的产量 vs 需求量',
        height=400,
        labels={'数量': '数量 (件)', '地点': '地点', '产品': '产品'}
    )
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig.show()

    # 另一个更清晰的方案：按产品切分
    fig2 = px.bar(
        production_demand_fine_df,
        x='周期（旬）',
        y='数量',
        color='类型',
        facet_row='产品', # 按产品切分
        facet_col='地点', # 按地点切分
        barmode='group',
        title='产量 vs 需求量（按产品和地点）',
        height=600,
        labels={'数量': '数量 (件)', '地点': '地点', '类型': '类型'}
    )
    fig2.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig2.show()

else:
    print("没有可展示的细化产量与需求数据。")


--- 工厂/客户产量与需求对照（细化到产品） ---


,地点,周期（旬）,产品,数量,类型
0,北京工厂,1,橙汁,32000.0,产量
1,北京工厂,1,芒果汁,21200.0,产量
2,广州工厂,1,橙汁,60000.0,产量
3,广州工厂,3,橙汁,46500.0,产量
4,广州工厂,1,苹果汁,59300.0,产量
...,...,...,...,...,...
65,杭州客户,2,葡萄汁,3800.0,需求量
66,杭州客户,3,葡萄汁,4200.0,需求量
67,杭州客户,1,芒果汁,3000.0,需求量
68,杭州客户,2,芒果汁,2800.0,需求量


## 最优生产计划

In [16]:
print("\n--- 最优生产计划 ---")
# 提取生产计划数据
production_plan_data = []
# 遍历所有生产相关的变量
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                # 只有当生产量大于0时才打印，忽略未生产的情况
                if value(produce[f][line][p][t]) > 0:
                    plan_entry = {
                        '工厂': f,
                        '生产线': line,
                        '产品': p,
                        '周期（旬）': t,
                        '生产量': int(value(produce[f][line][p][t])),
                        '单位生产成本': production_cost.get((f, line, p), 0)
                    }
                    production_plan_data.append(plan_entry)

production_plan_df = pd.DataFrame(production_plan_data)

if not production_plan_df.empty:
    # print(production_plan_df.to_string(index=False))
    display(production_plan_df)
else:
    print("没有生产记录。")

# 可视化3: 最优生产计划条形图
if not production_plan_df.empty:
    fig3 = px.bar(
        production_plan_df,
        x='工厂',
        y='生产量',
        color='产品',
        barmode='group',
        title='最优生产计划（各工厂生产量）',
        labels={'生产量': '生产量', '工厂': '工厂'}
    )
    fig3.show()


--- 最优生产计划 ---


,工厂,生产线,产品,周期（旬）,生产量,单位生产成本
0,北京工厂,Line_1,橙汁,1,32000,8.0
1,北京工厂,Line_2,芒果汁,1,21200,9.5
2,广州工厂,Line_A,橙汁,1,60000,7.5
3,广州工厂,Line_A,橙汁,3,46500,7.5
4,广州工厂,Line_A,芒果汁,2,29600,9.2
5,广州工厂,Line_B,苹果汁,1,59300,7.0
6,广州工厂,Line_B,苹果汁,3,34000,7.0
7,广州工厂,Line_B,葡萄汁,2,45000,8.5
8,上海工厂,Line_X,橙汁,2,27500,8.2
9,上海工厂,Line_X,葡萄汁,1,21000,8.8


## 最优配送方案

In [17]:
# --- 提取配送方案数据 ---
flow_data = []
# 工厂到分仓
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                if value(flow_fw[f][w][p][t]) > 0:
                    flow_data.append({
                        '起点': f,
                        '终点': w,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': int(value(flow_fw[f][w][p][t])),
                        '类型': '工厂->分仓'
                    })
# 工厂到客户
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_fc[f][c][p][t]) > 0:
                    flow_data.append({
                        '起点': f,
                        '终点': c,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': int(value(flow_fc[f][c][p][t])),
                        '类型': '工厂->客户'
                    })
# 分仓到客户
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_wc[w][c][p][t]) > 0:
                    flow_data.append({
                        '起点': w,
                        '终点': c,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': int(value(flow_wc[w][c][p][t])),
                        '类型': '分仓->客户'
                    })

flow_df = pd.DataFrame(flow_data)

# --- 1. 最优配送方案数据表格 ---
print("\n--- 最优配送方案 ---")
if not flow_df.empty:
    # print(flow_df.to_string(index=False))
    display(flow_df)
else:
    print("没有配送记录。")

# --- 2. 配送流向地图 ---
if not flow_df.empty:
    m = folium.Map(location=[34.7, 108.9], zoom_start=4)
    # 定义图标配置
    icon_configs = {
        'factories': {
            'icon': 'industry',
            'prefix': 'fa',
            'color': 'red',
            'icon_color': 'white',
            'tooltip': '工厂'
        },
        'warehouses': {
            'icon': 'warehouse',
            'prefix': 'fa',
            'color': 'blue',
            'icon_color': 'white',
            'tooltip': '分仓'
        },
        'customers': {
            'icon': 'user',
            'prefix': 'fa',
            'color': 'green',
            'icon_color': 'white',
            'tooltip': '客户'
        }
    }

    # 添加站点标记，并使用不同的图标
    marker_cluster = MarkerCluster().add_to(m)
    for name, coords in locations.items():
        # 根据站点类型选择图标配置
        if name in factories:
            config = icon_configs['factories']
        elif name in warehouses:
            config = icon_configs['warehouses']
        else: # 默认为客户
            config = icon_configs['customers']

        # 创建一个自定义图标
        custom_icon = folium.Icon(
            icon=config['icon'],
            prefix=config['prefix'],
            color=config['color'],
            icon_color=config['icon_color']
        )

        # 添加标记到地图，并指定自定义图标
        folium.Marker(
            location=coords,
            popup=name,
            tooltip=f"{name} ({config['tooltip']})",
            icon=custom_icon
        ).add_to(marker_cluster)

    # 定义产品颜色映射
    product_colors = {
        '橙汁': 'orange',
        '苹果汁': 'green'
    }

    # 最大的流量值，用于计算线的粗细
    max_flow = flow_df['数量'].max() if not flow_df.empty else 1

    # 新增：用于处理线条重叠的偏移字典
    overlap_offsets = defaultdict(int)

    # 按“旬”和“产品”两个维度创建 FeatureGroup
    for t in periods:
        for p in products:
            filtered_df = flow_df[(flow_df['周期（旬）'] == t) & (flow_df['产品'] == p)]

            if not filtered_df.empty:
                layer_name = f"第 {t} 旬 - {p}"
                product_period_layer = folium.FeatureGroup(name=layer_name).add_to(m)

                for _, row in filtered_df.iterrows():
                    source_coords_orig = locations[row['起点']]
                    dest_coords_orig = locations[row['终点']]

                    # 检查是否存在重叠，并获取偏移量
                    key = tuple(sorted([row['起点'], row['终点']])) # 使用排序后的元组作为键
                    current_offset = overlap_offsets[key]

                    # 定义偏移量，微调纬度和经度
                    offset_lat = current_offset * 0.05
                    offset_lon = current_offset * 0.05

                    source_coords = [source_coords_orig[0] + offset_lat, source_coords_orig[1] + offset_lon]
                    dest_coords = [dest_coords_orig[0] + offset_lat, dest_coords_orig[1] + offset_lon]

                    line_weight = row['数量'] / max_flow * 5 + 1
                    line_color = product_colors.get(row['产品'], 'gray')

                    path_points = [source_coords, dest_coords]
                    polyline = folium.PolyLine(path_points, color=line_color, weight=line_weight, opacity=0.8, tooltip=f"从 {row['起点']} 到 {row['终点']}").add_to(product_period_layer)

                    # AntPath(
                    #     [source_coords, dest_coords],
                    #     color=line_color,
                    #     weight=line_weight,
                    #     opacity=0.8,
                    #     pulse_color='transparent',
                    #     delay=400,
                    #     dash_array=[10, 20],
                    #     tooltip=f"从 {row['起点']} 到 {row['终点']}"
                    # ).add_to(product_period_layer)

                    mid_coords = [
                        (source_coords[0] + dest_coords[0]) / 2,
                        (source_coords[1] + dest_coords[1]) / 2
                    ]

                    description = f"""
                        <div style="white-space: nowrap; font-size: 10px; color: {line_color}; font-weight: bold;">
                            {row['产品']}<br>
                            {row['数量']:.0f}件
                        </div>
                    """

                    folium.Marker(
                        location=mid_coords,
                        icon=folium.DivIcon(html=description)
                    ).add_to(product_period_layer)
                    PolyLineTextPath(polyline, "▶", repeat=False, offset='100%', attributes={'fill': line_color, 'font-size': '15px'}).add_to(product_period_layer)

                    # 增加偏移计数器，为下一条重叠的线做准备
                    overlap_offsets[key] += 1

    folium.LayerControl().add_to(m)
    # 添加一个全屏按钮
    Fullscreen().add_to(m)

    m.save("supply_chain_flow_map.html")
    display(m)


--- 最优配送方案 ---


,起点,终点,产品,周期（旬）,数量,类型
0,北京工厂,上海分仓,芒果汁,1,12000,工厂->分仓
1,广州工厂,上海分仓,橙汁,1,15000,工厂->分仓
2,广州工厂,上海分仓,橙汁,3,16500,工厂->分仓
3,广州工厂,上海分仓,苹果汁,1,14000,工厂->分仓
4,广州工厂,上海分仓,苹果汁,2,15300,工厂->分仓
...,...,...,...,...,...,...
77,武汉分仓,成都客户,苹果汁,3,8000,分仓->客户
78,武汉分仓,成都客户,葡萄汁,2,4500,分仓->客户
79,武汉分仓,成都客户,葡萄汁,3,5500,分仓->客户
80,武汉分仓,成都客户,芒果汁,2,3500,分仓->客户


In [ ]:

# 提取流向数据用于地图
flow_data = defaultdict(list)
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                if value(flow_fw[f][w][p][t]) > 0:
                    flow_data['source'].append(f)
                    flow_data['destination'].append(w)
                    flow_data['product'].append(p)
                    flow_data['period'].append(t)
                    flow_data['quantity'].append(value(flow_fw[f][w][p][t]))
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_fc[f][c][p][t]) > 0:
                    flow_data['source'].append(f)
                    flow_data['destination'].append(c)
                    flow_data['product'].append(p)
                    flow_data['period'].append(t)
                    flow_data['quantity'].append(value(flow_fc[f][c][p][t]))
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_wc[w][c][p][t]) > 0:
                    flow_data['source'].append(w)
                    flow_data['destination'].append(c)
                    flow_data['product'].append(p)
                    flow_data['period'].append(t)
                    flow_data['quantity'].append(value(flow_wc[w][c][p][t]))

flow_df = pd.DataFrame(flow_data)

# 可视化2: 供应链流向地图
if not flow_df.empty:
    m = folium.Map(location=[34.7, 108.9], zoom_start=4)
    for name, coords in locations.items():
        folium.Marker(coords, popup=name, tooltip=name).add_to(m)

    max_flow = flow_df['quantity'].max()
    for _, row in flow_df.iterrows():
        source_coords = locations[row['source']]
        dest_coords = locations[row['destination']]
        line_weight = row['quantity'] / max_flow * 5 + 1
        folium.PolyLine(
            [source_coords, dest_coords],
            color='blue',
            weight=line_weight,
            opacity=0.7,
            tooltip=f"从 {row['source']} 到 {row['destination']}: {row['quantity']:.0f} 件 {row['product']} (第 {row['period']} 旬)"
        ).add_to(m)
    display(m)
else:
    print("没有可展示的流向数据。")